<a href="https://colab.research.google.com/github/girish445ai/Deep-learning-Assignment-1-/blob/main/Convolutional_Neural_Networks_PART_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install wandb

In [2]:
import numpy as np
import keras 
import tensorflow as tf
import pandas as pd
import wandb
import matplotlib.pyplot as plt
from tensorflow.keras import layers,models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from wandb.keras import WandbCallback

In [ ]:
!curl -SL https://storage.googleapis.com/wandb_datasets/nature_12K.zip > nature_12K.zip
!unzip nature_12K.zip

train_datapath='inaturalist_12K/train/'
test_datapath='inaturalist_12K/val/'
categories=['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']

In [4]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = train_datapath,
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      batch_size = 32,
                      image_size = (256, 256),
                      shuffle = True,
                      seed = 42,
                      validation_split = 0.2,
                      subset = 'training')

val_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = train_datapath,
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      batch_size = 32,
                      image_size = (256, 256),
                      color_mode = 'rgb',
                      shuffle = True,
                      seed = 42,
                      validation_split = 0.2,
                      subset = 'validation')

NameError: ignored

In [ ]:
# 20% validation data
len_train, len_val = len(train_data), len(val_data)
train_data = train_data.take(int(0.2*len_train))
val_data = val_data.take(int(0.2*len_val))

In [ ]:
train_data_aug = ImageDataGenerator(
                            rescale = None,
                            rotation_range = 20,
                            width_shift_range = 0.2,
                            height_shift_range = 0.2,
                            shear_range = 0.2,
                            zoom_range = 0.2,
                            horizontal_flip=True,
                            data_format = 'channels_last',
                            validation_split = 0.2)        #Specifying parameters for augmentation of training data

val_data_aug = ImageDataGenerator(validation_split = 0.2) #No augmentation of validation data

train_aug_gen = train_data_aug.flow_from_directory(train_datapath, shuffle = True, \
                                                         seed = 42, subset = 'training')
val_aug_gen = val_data_aug.flow_from_directory(train_datapath, shuffle = True, \
                                                     seed = 42, subset = 'validation')

train_aug_data = tf.data.Dataset.from_generator(
                    lambda: train_aug_gen,
                    output_types = (tf.float32, tf.float32),
                    output_shapes = ([None, 256, 256, 3], [None, 10]))

val_aug_data = tf.data.Dataset.from_generator(
                  lambda: val_aug_gen,
                  output_types = (tf.float32, tf.float32),
                  output_shapes = ([None, 256, 256, 3], [None, 10]))

In [ ]:
# 20% validation data
len_train_aug, len_val_aug = len(train_aug_data), len(val_aug_data)
train_aug_data = train_aug_data.take(int(0.2*len_train_aug))
val_aug_data = val_aug_data.take(int(0.2*len_val_aug))

In [ ]:
def cnn_train(config=sweep_config):
    with wandb.init(config=config):
        config = wandb.init().config
        wandb.run.name = 'firstLayerFilters_{}_filterOrg_{}_dataAug_{}_batchNorm_{}_dropout_{}_kerSize_{}_denseSize_{}'\
                         .format(config.first_layer_filters, config.filter_org, config.data_aug, config.batch_norm, config.dropout, config.kernel_size, config.dense_size)
        
        inputs = tf.keras.Input(shape = (256, 256, 3))
        mod = tf.keras.layers.Rescaling(scale = 1.0/255)(inputs)
        filter_sizes = [int(config.first_layer_filters*(config.filter_org**layer_num)) for layer_num in range(config.conv_layers)]
        ker_size = config.kernel_size

        # Apply some convolution and pooling layers
        for layer_num in range(config.conv_layers):
            mod = layers.Conv2D(filters = filter_sizes[layer_num], kernel_size = (ker_size, ker_size))(mod)
            if config.batch_norm:
                mod = layers.BatchNormalization(axis = -1)(mod)
            mod = layers.Activation(config.activation)(mod)
            mod = layers.MaxPooling2D(pool_size = (2, 2))(mod)            
                
        # Dense Layer
        mod = layers.Flatten()(x)
        mod = layers.Dense(config.dense_size)(x)
        if config.batch_norm:
            mod = layers.BatchNormalization(axis = -1)(x)
        mod = layers.Activation(config.activation)(x)
        if config.dropout > 0:
            mod = layers.Dropout(rate = config.dropout)(x)        

        # Output Layer
        outputs = layers.Dense(10, activation ='softmax')(mod)
        model = tf.keras.Model(inputs = inputs, outputs = outputs)

        sweep_val_data = val_data # In any case, validation data is not augmented
        if config.data_aug == 1:
          sweep_train_data = train_aug_data
        else:
          sweep_train_data = train_data


        model.compile(optimizer=config.optimizer,
                      loss = tf.keras.losses.CategoricalCrossentropy(name = 'loss'),
                      metrics = [tf.keras.metrics.CategoricalAccuracy(name = 'acc')])
        
        model_hist = model.fit(sweep_train_data, epochs = config.num_epochs,
                               validation_data = sweep_val_data, 
                               callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 5), wandb.keras.WandbCallback()])

In [14]:
wandb.init()

sweep_config = {
    'method': 'random', #grid, random, bayes
    'metric': {
      'name': 'val_acc',
      'goal': 'maximize'
      },
    'parameters': {
        'activation': {
            'values': ['relu','leaky_relu']
        },
        'kernel_size': {
            'values': [3, 5, 7] 
        },
                
        'filter_org': {
            'values': [0.5, 1, 2]
        },

        'dense_size': {
            'values': [32, 64, 128]
        },

        'first_layer_filters': {
            'values': [32, 64]
        },

        'conv_layers': {
            'values': [5]
        },

        'dropout': {
            'values': [0.2, 0.3]
        },

        'num_epochs': {
            'values': [50]
        },

        'batch_norm': {
            'values': [True, False]
        },

        'optimizer': {
            'values': ['adam']
        },

        'data_aug': {
            'values': [True, False]
        },
    }
}

sweep_id = wandb.sweep(sweep_config, project = 'assgn_jy2')
wandb.agent(sweep_id, function=cnn_train)

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: hdkj17h2
Sweep URL: https://wandb.ai/jyothiraditya/assignment2/sweeps/hdkj17h2


...

wandb: Agent Starting Run: 3qkofalg with config:
wandb: 	F_size: 3
wandb: 	augmentation: True
wandb: 	batch_norm: True
wandb: 	dropout: 0.2
wandb: 	layer1: 16
wandb: 	layer2: 32
wandb: 	layer3: 64
wandb: 	layer4: 128
wandb: 	layer5: 256


wandb: Agent Starting Run: jag97viw with config:
wandb: 	F_size: 3
wandb: 	augmentation: False
wandb: 	batch_norm: True
wandb: 	dropout: 0.2
wandb: 	layer1: 16
wandb: 	layer2: 32
wandb: 	layer3: 64
wandb: 	layer4: 128
wandb: 	layer5: 256


wandb: Agent Starting Run: ctzxy1ty with config:
wandb: 	F_size: 3
wandb: 	augmentation: True
wandb: 	batch_norm: True
wandb: 	dropout: 0.2
wandb: 	layer1: 16
wandb: 	layer2: 32
wandb: 	layer3: 64
wandb: 	layer4: 128
wandb: 	layer5: 256


wandb: Ctrl + C detected. Stopping sweep.


<Figure size 432x288 with 0 Axes>

In [ ]:
test_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = test_datapath,
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      batch_size = 32,
                      image_size = (256, 256),
                      shuffle = False,
                      seed = 42,
                      validation_split = 0)